# Automated Anomaly Detection
**Objective**: Understand and practice automated anomaly detection using various techniques.

**Task**: Autoencoders for Anomaly Detection

**Steps**:
1. Data Set: Download a dataset of electricity consumption data.
2. Build an Autoencoder: Construct a simple autoencoder using a neural network for the
normal consumption data.
3. Identify Anomalies: Use the trained model to reconstruct the data and identify anomalies based on reconstruction error.
4. Visualize: Plot both the actual and reconstructed data to highlight anomalies.

In [2]:
# write your code from here

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# Ensure tensorflow is installed: pip install tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import requests
import io
url="https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/ambient_temperature_system_failure.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))
df['timestamp']=pd.to_datetime(df['timestamp'])
df.set_index('timestamp',inplace=True)
data=df['value'].values.reshape(-1,1)
scaler=MinMaxScaler()
data_scaled=scaler.fit_transform(data)
normal_data_threshold=1.5
mean_data_scaled=np.mean(data_scaled)
std_data_scaled=np.std(data_scaled)
normal_data_indices=np.where(np.abs(data_scaled-mean_data_scaled)<normal_data_threshold*std_data_scaled)[0]
normal_data=data_scaled[normal_data_indices]
X_train,X_test=train_test_split(normal_data,test_size=0.2,random_state=42)
input_dim=X_train.shape[1]
latent_dim=1
input_layer=Input(shape=(input_dim,))
encoder=Dense(latent_dim,activation='relu')(input_layer)
decoder=Dense(input_dim,activation='sigmoid')(encoder)
autoencoder=Model(inputs=input_layer,outputs=decoder)
autoencoder.compile(optimizer=Adam(learning_learning_rate=0.001),loss='mse')
history=autoencoder.fit(X_train,X_train,epochs=50,batch_size=32,shuffle=True,validation_data=(X_test,X_test),verbose=0)
reconstructions=autoencoder.predict(data_scaled)
mse=np.mean(np.power(data_scaled-reconstructions,2),axis=1)
threshold=np.mean(mse)+3*np.std(mse)
anomalies=df[mse>threshold]
plt.figure(figsize=(15,7))
plt.plot(df.index,data_scaled,label='Actual Data')
plt.plot(df.index,reconstructions,label='Reconstructed Data',alpha=0.7)
plt.scatter(anomalies.index,scaler.transform(anomalies['value'].values.reshape(-1,1)),color='red',label='Anomaly',s=50,zorder=5)
plt.title('Anomaly Detection using Autoencoder on Electricity Consumption Data')
plt.xlabel('Timestamp')
plt.ylabel('Scaled Value')
plt.legend()
plt.grid(True)
plt.show()

ModuleNotFoundError: No module named 'tensorflow'